Assignment 3 - Diachronic analysis of movie dialogues
===

*Due: January 1 2023*

In this assignment you will analyze how the expression of Hope in movie dialogues changed with time.

## Imports

In [245]:
import os
from nltk.corpus import wordnet as wn
import pandas as pd
import ipywidgets as widgets
import itertools
import nltk
import regex as re
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models.word2vec import Word2Vec
import ast
from glob import glob
import collections


lemma = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))
layout = widgets.Layout(width="auto")
lang = "eng"
data_path = "data/dialogs_preprocessed2/"

## Functions

### Functions for text preprocessing

In [94]:
def clean_url(input):
    output = re.sub(r"http\S+", "", input)
    return output


def fix_contraction(input):
    output = contractions.fix(input)
    return output


def clean_non_alphanumeric(input):
    output = re.sub(r"[^a-zA-Z0-9]", " ", input)
    return output


def clean_tokenization(input):
    output = nltk.word_tokenize(input)
    return output


def clean_stopwords(input):
    output = [item for item in input if item not in stop_words]
    return output


def numbers_to_words(input):
    output = []
    for item in input:
        if item.isnumeric() == True:
            output += [p.number_to_words(item)]
        else:
            output += [item]
    return output


def clean_lowercase(input):
    output = str(input).lower()
    return output


def clean_lemmatization(input):
    output = [lemma.lemmatize(word=w, pos="v") for w in input]
    return output


def clean_length(input):
    output = [word for word in input if len(word) > 2]
    return output


def convert_to_string(input):
    output = " ".join(input)
    return output


def preprocessing(text, remove_stopwords=True):
    """
    Preprocessing pipeline.
    """
    text = clean_url(text)
    text = fix_contraction(text)
    text = clean_non_alphanumeric(text)
    text = clean_lowercase(text)
    text = clean_tokenization(text)
    text = numbers_to_words(text)
    if remove_stopwords:
        text = clean_stopwords(text)
    text = clean_lemmatization(text)
    text = clean_length(text)
    text = convert_to_string(text)
    return text

### Functions for wordnet

In [80]:
def generate_word_list(seed_word, language):
    """
    Takes in a single seed word and returns
    a word list, the length and a list of synsets
    """

    ## we create an empty list to store the final word list
    list_of_lemmas = []
    list_of_meanings = []

    ## a function to add a word to a list
    add_to_list = lambda list1, item1: list1.append(item1)

    ## a function to return the hyponyms of a synset
    hypos = lambda s: s.hyponyms()

    ## wn.synset obtains the list of synonyms and meanings for that word, in different syntactic categories
    meanings = wn.synsets(seed_word, pos=wn.NOUN + wn.VERB + wn.ADJ)

    ## loop over set of meanings in synset
    for meaning in meanings:

        ## add synset, definition, and a list of all associated lemmas into the list_of_meanings
        list_of_meanings += [
            [
                meaning,
                meaning.definition(),
                [lemma.name() for lemma in meaning.lemmas(language)],
            ]
        ]

        ## append all synonyms (lemmas()) of that meaning to the list_of_lemmas
        [
            add_to_list(list_of_lemmas, lemma.name())
            for lemma in meaning.lemmas(language)
        ]

        ## loop over the list of all possible hyponyms
        for hyponym in meaning.closure(hypos):

            ## add synsets, definition, and a list of all associated lemmas into the list_of_meanings
            list_of_meanings += [
                [
                    hyponym,
                    hyponym.definition(),
                    [lemma.name() for lemma in hyponym.lemmas(language)],
                ]
            ]

            ## append all synonyms (lemmas()) of that hyponym to the list_of_lemmas
            [
                add_to_list(list_of_lemmas, lemma.name())
                for lemma in hyponym.lemmas(language)
            ]

    ##eliminate list duplications by applying the set transformation
    set_of_lemmas = [*set(list_of_lemmas)]

    ## sort alphabetically
    set_of_lemmas.sort()

    ##length
    length = len(set_of_lemmas)

    return (set_of_lemmas, length, list_of_meanings)


def prune_list(w, syn_list):
    """
    Takes in a container checkbox widget,
    a list with synsets and returns
    a filtered word list.
    """

    filtered_meanings = list(
        itertools.compress(syn_list, [widget.value for widget in w.children])
    )

    filtered_list = [word for lemmas in filtered_meanings for word in lemmas[2]]

    filtered_list = sorted([*set(filtered_list)])

    return filtered_list


def expand_meanings(seed_words, language):
    """
    Takes in a list of seed words and returns all synsets.
    """

    list_meanings = []

    for sw in seed_words:
        _, _, meanings = generate_word_list(sw, language)
        list_meanings += meanings

    list_meanings.sort()

    # groupby also eliminates duplications
    list(list_meanings for list_meanings, _ in itertools.groupby(list_meanings))

    return list_meanings

### Functions for vec2word

In [181]:
def create_word2vec_input(root_folder):
    """

    Function to select .txt files and store them as a list of paragraphs,
    each a list of words, to use as input to the function WordVec.

    This function was originally named "literary_words_list".

    Parameters
    ----------
    root_folder : a file path where .txt files are located
    e.g. '~/home/user/text_analyses'

    Returns
    -------
    A list of the paragraphs, each paragraph a list of words

    """

    word2vec_input = []

    for path, subdirs, files in os.walk(root_folder):
        for file in files:
            if ".txt" in file and "model" not in file:
                # print(file)
                name = os.path.join(path, file)
                file_text = open(name, encoding="utf-8").read()
                text_list_paragraphs = file_text.split("\n")

                for paragraph in text_list_paragraphs:
                    paragraph = paragraph.replace("\r", "")
                    list_of_sentences = nltk.sent_tokenize(paragraph)

                    for sentence in list_of_sentences:
                        word2vec_input += [preprocessing(sentence)]

    # remove empty strings and split sentences into lists
    word2vec_input = [s.split() for s in word2vec_input if s != ""]

    return word2vec_input


def get_word2vec_list(word_list, model):
    """
    Function to use word2vec to inquiry about the 10 most similar semantically
    words to each seed word in word_list. Uses a pre-trained model to get the
    most similar words within the text.

    """

    list_of_word2vec_lists = []
    for word in word_list:
        try:

            ##
            list_vects = model.wv.most_similar([word], topn=10)

            new_list = []
            new_list += [word]
            for item in list_vects:
                word1 = item[0]
                new_list += [word1]

            # print(new_list)
            # print('\n')
            list_of_word2vec_lists += [new_list]

        except KeyError:
            continue

    return list_of_word2vec_lists

### Functions for word frequency

In [334]:
def compute_frequencies(list_string, list_of_categories):
    """
    Function to calculate word frequencies.

    """

    global apply_counter1

    tokenized_text = ast.literal_eval(list_string)
    word_counter = collections.Counter(tokenized_text)
    total_words = len(tokenized_text)

    list_of_category_frequencies = []
    for category in list_of_categories:
        category_count = 0.0

        for word in category:
            category_count += int(word_counter[word])

        if total_words != 0 and category_count > 0:
            category_frequency = category_count / total_words
        else:
            category_frequency = float("nan")
        list_of_category_frequencies += [category_frequency]

    apply_counter1 += 1

    return list_of_category_frequencies

## 0. Expression of interest: Hope

In [168]:
seed_word = "hope"

## 1. Control measure: Irritation

Find control measure to contrast with "Hope"

In [169]:
ctrl_seed_word = "irritation"

## 2. Initial bags of seeds

Find psychometric tools or equivalent to generate initial bags of seeds, for both Hope and the control condition

In [170]:
seed_list = generate_word_list(seed_word, lang)
print(f"Length of lemma list is {seed_list[1]}")

Length of lemma list is 16


In [171]:
ctrl_seed_list = generate_word_list(control_seed_word, lang)
print(f"Length of lemma list is {ctrl_seed_list[1]}")

Length of lemma list is 29


## 3. Expand bag of seeds and get synonyms and hyponyms

Use WordNet tools to expand your bag of seeds and get synonyms and hyponyms, and start excluding words with unrelated meanings using the filters available.

### Hope

In [172]:
selection_widget = widgets.VBox(
    [
        widgets.Checkbox(
            value=True,
            description=str(item),
            disabled=False,
            indent=False,
            layout=layout,
        )
        for item in seed_list[2]
    ]
)

selection_widget

In [173]:
seed_words = prune_list(selection_widget, seed_list[2])

print(seed_words)

['desire', 'encouragement', 'go_for', 'great_white_hope', 'hope', 'hopefulness', 'optimism', 'promise', 'rainbow', 'sanguineness', 'sanguinity', 'trust', 'white_hope']


#### Extend

In [174]:
synsets = expand_meanings(seed_words, lang)
len(synsets)

300

In [175]:
synsets[:3]

[[Synset('abetment.n.01'),
  'the verbal act of urging on',
  ['abetment', 'abettal', 'instigation']],
 [Synset('accept.v.03'),
  'give an affirmative reply to; respond favorably to',
  ['accept', 'consent', 'go_for']],
 [Synset('accept.v.07'),
  'tolerate or accommodate oneself to',
  ['accept', 'live_with', 'swallow']]]

### Irritation

In [176]:
ctrl_selection_widget = widgets.VBox(
    [
        widgets.Checkbox(
            value=True,
            description=str(item),
            disabled=False,
            indent=False,
            layout=layout,
        )
        for item in ctrl_seed_list[2]
    ]
)

ctrl_selection_widget

In [177]:
ctrl_seed_words = prune_list(ctrl_selection_widget, ctrl_seed_list[2])

print(ctrl_seed_words)

['aggravation', 'aggro', 'annoyance', 'annoying', 'botheration', 'bummer', 'discomfort', 'exacerbation', 'exasperation', 'excitation', 'huff', 'impatience', 'innervation', 'irritation', 'last_straw', 'miff', 'pinprick', 'pique', 'provocation', 'red_flag', 'restlessness', 'seeing_red', 'snit', 'soreness', 'taunt', 'taunting', 'temper', 'twit', 'vexation']


#### Extend

In [178]:
ctrl_synsets = expand_meanings(ctrl_seed_words, lang)
len(ctrl_synsets)

208

In [179]:
ctrl_synsets[:3]

[[Synset('abatable_nuisance.n.01'),
  'a nuisance that can remedied (suppressed or extinguished or rendered harmless)',
  ['abatable_nuisance']],
 [Synset('abatable_nuisance.n.01'),
  'a nuisance that can remedied (suppressed or extinguished or rendered harmless)',
  ['abatable_nuisance']],
 [Synset('aggravation.n.01'),
  'an exasperated feeling of annoyance',
  ['aggravation', 'exasperation']]]

## 4. Train semantic vector space model

Train a semantic vector space model using word2vec

### Create input for vec2word from folder with textfiles

In [183]:
word2vec_input = create_word2vec_input(data_path)

### Train and save the model

In [184]:
word2vec_output = Word2Vec(word2vec_input_list, min_count=1)

## Save vector space
word2vec_output.save("data/w2v_model.txt")

### Load the model

In [185]:
model = Word2Vec.load("data/w2v_model.txt")

## 5. Semantic clouds

Use your vector space model to find out the semantic clouds of each word in your bag of seeds, and select only the words with semantically meaningful clouds

### Hope

In [207]:
vector_space = get_word2vec_list(filtered_list, model)

In [216]:
filtered_list[5]

'hopefulness'

In [213]:
vector_space[3]

['hopefulness',
 'sonofabitch',
 'durn',
 'blighter',
 'cunt',
 'buzzard',
 'motherfucker',
 'sonovabitch',
 'faggot',
 'cocksucker',
 'puttin']

### Irritation

In [194]:
ctrl_vector_space = get_word2vec_list(ctrl_filtered_list, model)

In [206]:
ctrl_filtered_list[5]

'bummer'

In [203]:
ctrl_vector_space[3]

['bummer',
 'hangover',
 'stinker',
 'weenie',
 'fruity',
 'fixer',
 'cheerleader',
 'libel',
 'disorientation',
 'schoolgirl',
 'turf']

## 6. Final bag of words and word frequencies

Take the final Bag of Words, prune it further if necessary, and use the second script to calculate word frequencies (don't forget to upload the text files)

### Hope

In [218]:
layout = widgets.Layout(width="auto")

selection_widget_2 = widgets.VBox(
    [
        widgets.Checkbox(
            value=True,
            description=str(item),
            disabled=False,
            indent=False,
            layout=layout,
        )
        for item in vector_space
    ]
)

selection_widget_2

In [ ]:
filtered_word2vec = list(
    itertools.compress(
        vector_space, [widget.value for widget in selection_widget_2.children]
    )
)

## now we extract just the words
flat_list = [word for lists in filtered_word2vec for word in lists]

## eliminate duplications, and sort alphabetically
flat_list = sorted([*set(flat_list)])

len(flat_list)

75

And a final check

In [228]:
selection_widget_3 = widgets.VBox(
    [
        widgets.Checkbox(
            value=True,
            description=str(item),
            disabled=False,
            indent=False,
            layout=layout,
        )
        for item in flat_list
    ]
)

selection_widget_3

In [229]:
final_list = list(
    itertools.compress(
        flat_list, [widget.value for widget in selection_widget_3.children]
    )
)

len(final_list)

41

### Irritation

In [219]:
ctrl_selection_widget_2 = widgets.VBox(
    [
        widgets.Checkbox(
            value=True,
            description=str(item),
            disabled=False,
            indent=False,
            layout=layout,
        )
        for item in ctrl_vector_space
    ]
)

ctrl_selection_widget_2

In [227]:
ctrl_filtered_word2vec = list(
    itertools.compress(
        ctrl_vector_space, [widget.value for widget in ctrl_selection_widget_2.children]
    )
)

## now we extract just the words
ctrl_flat_list = [word for lists in ctrl_filtered_word2vec for word in lists]

## eliminate duplications, and sort alphabetically
ctrl_flat_list = sorted([*set(ctrl_flat_list)])

len(ctrl_flat_list)

192

And a final check

In [230]:
ctrl_selection_widget_3 = widgets.VBox(
    [
        widgets.Checkbox(
            value=True,
            description=str(item),
            disabled=False,
            indent=False,
            layout=layout,
        )
        for item in ctrl_flat_list
    ]
)

ctrl_selection_widget_3

In [231]:
ctrl_final_list = list(
    itertools.compress(
        ctrl_flat_list, [widget.value for widget in ctrl_selection_widget_3.children]
    )
)

len(ctrl_final_list)

81

In [232]:
hope = final_list
irritation = ctrl_final_list

list_of_categories = [hope, irritation]

### Frequencies

<div class="alert alert-warning">Is this correct? I was not sure what does the input need to look like. And from where does it need to come from? A string that looks like a list of strings???</div>

In [328]:
df = pd.DataFrame(glob(data_path + "/*.txt"), columns=["movie"])
df["path"] = df["movie"]
df["movie"] = (
    df["movie"]
    .str.replace("^" + data_path, "", regex=True)
    .str.replace("_dialog.txt$", "", regex=True)
)


df["text"] = df["path"].apply(lambda path: open(path, encoding="utf-8").read())
df["text"] = df["text"].apply(lambda raw_txt: preprocessing(raw_txt))
df["text"] = df["text"].apply(lambda txt: str(txt.split(" ")))

df.head()

,movie,path,text
0,donniebrasco,data/dialogs_preprocessed2/donniebrasco_dialog...,"['paul', 'attanasio', 'base', 'book', 'donnie'..."
1,shiningthe,data/dialogs_preprocessed2/shiningthe_dialog.txt,"['post', 'production', 'script', 'july', 'get'..."
2,idesofmarchthe,data/dialogs_preprocessed2/idesofmarchthe_dial...,"['write', 'george', 'clooney', 'grant', 'heslo..."
3,hangoverthe,data/dialogs_preprocessed2/hangoverthe_dialog.txt,"['write', 'jon', 'lucas', 'scott', 'moore', 's..."
4,bringingoutthedead,data/dialogs_preprocessed2/bringingoutthedead_...,"['first', 'draft', 'paul', 'schrader', 'novel'..."


In [352]:
apply_counter1 = 0

df["frequencies"] = df["text"].apply(
    compute_frequencies, list_of_categories=list_of_categories
)

# Here we distribute the list of frequencies of different categories to different columns

# first we create a list for each category ...
hope2 = []
irritation2 = []
list_of_categories2 = [hope2, irritation2]


for index, row in df.iterrows():
    list_of_frequencies = row["frequencies"]

    for category, frequency in zip(list_of_categories2, list_of_frequencies):
        category += [frequency]

# ... and then we write each list to a column
list_of_category_names = ["hope", "irritation"]

for catName, catFreqs in zip(list_of_category_names, list_of_categories2):
    df[catName] = catFreqs

del df["frequencies"]

df.head()

,movie,path,text,hope,irritation,ratio_hope_irritation
0,donniebrasco,data/dialogs_preprocessed2/donniebrasco_dialog...,"['paul', 'attanasio', 'base', 'book', 'donnie'...",0.002886,0.000462,0.724138
1,shiningthe,data/dialogs_preprocessed2/shiningthe_dialog.txt,"['post', 'production', 'script', 'july', 'get'...",0.007365,0.000263,0.931034
2,idesofmarchthe,data/dialogs_preprocessed2/idesofmarchthe_dial...,"['write', 'george', 'clooney', 'grant', 'heslo...",0.004899,0.000639,0.769231
3,hangoverthe,data/dialogs_preprocessed2/hangoverthe_dialog.txt,"['write', 'jon', 'lucas', 'scott', 'moore', 's...",0.004562,0.000480,0.809524
4,bringingoutthedead,data/dialogs_preprocessed2/bringingoutthedead_...,"['first', 'draft', 'paul', 'schrader', 'novel'...",0.002922,0.000674,0.625000


Save dataframe

In [353]:
df.to_csv("data/hope_irritation_frequencies.csv")

## 7. Relevant ratio, visualization of time series

Calculate your psychological relevant ratio, build a dataframe and plot a time series using Seaborn

### Ratio hope vs. irritation

<div class="alert alert-warning">What kind of ratio is this (relative difference)?</div>

In [354]:
df["ratio_hope_irritation"] = (df["hope"] - df["irritation"]) / (
    df["hope"] + df["irritation"]
)

df.head()

,movie,path,text,hope,irritation,ratio_hope_irritation
0,donniebrasco,data/dialogs_preprocessed2/donniebrasco_dialog...,"['paul', 'attanasio', 'base', 'book', 'donnie'...",0.002886,0.000462,0.724138
1,shiningthe,data/dialogs_preprocessed2/shiningthe_dialog.txt,"['post', 'production', 'script', 'july', 'get'...",0.007365,0.000263,0.931034
2,idesofmarchthe,data/dialogs_preprocessed2/idesofmarchthe_dial...,"['write', 'george', 'clooney', 'grant', 'heslo...",0.004899,0.000639,0.769231
3,hangoverthe,data/dialogs_preprocessed2/hangoverthe_dialog.txt,"['write', 'jon', 'lucas', 'scott', 'moore', 's...",0.004562,0.000480,0.809524
4,bringingoutthedead,data/dialogs_preprocessed2/bringingoutthedead_...,"['first', 'draft', 'paul', 'schrader', 'novel'...",0.002922,0.000674,0.625000


### Time series

<div class="alert alert-warning">For this we would need the years of the movies</div>

In [355]:
#
#
#